In [7]:
import os
import sys
import random
import datetime
import numpy as np
import uuid
import time
import json
import copy

from models import Models
#from tools.utils import *
from config import Config

from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import svm
from dataset import load_data

In [4]:
name = "books"
encoding='utf-8'
test_set, _ = load_data(os.path.join('dataset', 'raw', name, name+'.task.test'), encoding=encoding)

In [30]:
x,y=link_words(test_set)

In [29]:
def link_words(samples):
    '''link sentence back
    return x, y
    '''
    x, y = list(), list()
    # line: (sentence, label)
    for line in samples:
        x.append(' '.join(line[0]))
        y.append(line[1])
    vectorizer = CountVectorizer(ngram_range=(1,3))
    x = vectorizer.fit_transform(x).toarray()
    return x, y

In [16]:
len(x[0])

37067

In [25]:
len(x[0])

7912

In [22]:
len(x[0])

44979

In [28]:
len(x[0])

52022

In [31]:
len(x[0])

97001

In [15]:
def prepro_bow(name, filename, ngram, encoding='utf-8'):
    print('Process {} dataset...'.format(name))
    word_count = {}
    def link_words(samples):
        '''link sentence back
        return x, y
        '''
        x, y = list(), list()
        # line: (sentence, label)
        for line in samples:
            sentence = line[0]
            for w in sentence:
                if w in word_count:
                    word_count[w]+=1
                else:
                    word_count[w]=0
        for line in samples:
            sentence = []
            for w in line[0]:
                if word_count[w] >= 20:
                    sentence.append(w)
            x.append(' '.join(sentence))
            y.append(line[1])
        vectorizer = CountVectorizer(ngram_range=(1,ngram))
        x = vectorizer.fit_transform(x).toarray()
        return x, y

    # load dataset
    train_set, _ = load_data(os.path.join('dataset', 'raw', name, filename+'_train.txt'), encoding=encoding)
    #dev_set, _ = load_data(os.path.join('dataset', 'raw', 'books', 'books.task.dev'))
    test_set, _ = load_data(os.path.join('dataset', 'raw', name, filename+'_test.txt'), encoding=encoding)
    #train_set.extend(dev_set)
    train_len = len(train_set)
    train_set.extend(test_set)
    x_train, y_train = link_words(train_set)
    x_test, y_test = x_train[train_len:], y_train[train_len:]
    x_train, y_train = x_train[:train_len], y_train[:train_len]
    print('Process {} dataset done'.format(name))
    return x_train, y_train, x_test, y_test


def single_task(task, model_name, ngram=1):
    def score(y_pred, y):
        assert len(y_pred) == len(y), 'y_pred and y have different lenght'
        acc = 0
        for y1, y2 in zip(y_pred, y):
            if y1 == y2:
                acc += 1
        return float(acc) / len(y)
    if model_name == 'Bayes':
        clf = MultinomialNB()
    elif model_name == 'svm':
        clf = svm.SVC(gamma='scale')

    x_train, y_train, x_test, y_test = prepro_bow(task, task,ngram)

    clf.fit(x_train, y_train)
    #print('fit done')
    
    print(cross_val_score(clf, x_train, y_train, scoring='accuracy', cv=10))

    #print('Accuracy: {}'.format(score(clf.predict(x_test), y_test)))

In [12]:
single_task('imdb', 'Bayes')

Process imdb dataset...
Process imdb dataset done
[0.8488 0.844  0.8364 0.8312 0.8404 0.8464 0.844  0.858  0.8428 0.842 ]


In [ ]:
single_task('imdb', 'Bayes', 2)

Process imdb dataset...
Process imdb dataset done


In [ ]:
single_task('imdb', 'Bayes', 3)

In [42]:
single_task('books', 'Bayes')

Process books dataset...
[0.71428571 0.86956522 0.80625    0.725      0.8375     0.7625
 0.8        0.74375    0.79245283 0.77358491]


In [44]:
single_task('books', 'Bayes')

Process books dataset...
[0.71428571 0.8136646  0.80625    0.73125    0.8125     0.8
 0.7625     0.80625    0.79874214 0.81132075]


In [46]:
single_task('books', 'Bayes')#1,2

Process books dataset...
[0.74534161 0.86335404 0.8125     0.75       0.86875    0.80625
 0.78125    0.7625     0.81132075 0.79245283]


In [48]:
single_task('books', 'Bayes')#3,3

Process books dataset...
[0.72049689 0.74534161 0.71875    0.73125    0.75       0.7125
 0.7125     0.7        0.67924528 0.77987421]


In [50]:
single_task('books', 'Bayes')#1,3

Process books dataset...
[0.76397516 0.86956522 0.8375     0.76875    0.85625    0.8
 0.80625    0.7875     0.82389937 0.79245283]


In [52]:
single_task('books', 'Bayes')#1,4

Process books dataset...
[0.76397516 0.88198758 0.8375     0.7875     0.8375     0.8125
 0.8125     0.8125     0.79245283 0.81132075]


In [55]:
def getall(name, encoding='utf-8'):
    filename = name
    train_set, _ = load_data(os.path.join('dataset', 'raw', name, filename+'.task.train'), encoding=encoding)
    #dev_set, _ = load_data(os.path.join('dataset', 'raw', name, filename+'.task.dev'), encoding=encoding)
    test_set, _ = load_data(os.path.join('dataset', 'raw', name, filename+'.task.test'), encoding=encoding)
    #train_set.extend(dev_set)
    train_set.extend(test_set)
    train_len = len(train_set)
    return train_set, train_len

In [66]:
a,b = getall('kitchen')

In [67]:
d = dict()
for i, j in a:
    if j not in d:
        d[j]=1
    else:
        d[j]+=1

In [68]:
b

2000

In [69]:
d

{1: 1000, 0: 1000}

In [ ]:
dataset_meta_data = {
    'books': [1000, 1000],
    'electronics': [1000, 1000],
    'kitchen': [1000, 1000],
}